In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras.layers import Dense, Dropout, Flatten, Input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras import applications

Using TensorFlow backend.


In [2]:
# from keras import backend as K
# K.set_image_dim_ordering('tf')

In [3]:
path = 'C:/Users/Isaac/Documents/tensorflow/cat_dog/subset/'

In this tutorial we will use vgg16 pre-trained model for classification. First download model weights of top layers(before FC layers)and then train a classifier on top of that. First use top layers(all conv blocks) of vgg16(model1) and pass our images through that and get the features. Then use these features as input for our classifier model(model2).

In [4]:
batch_size = 16
nb_train_samples = 2000
nb_validation_samples = 800

In [5]:
model1 = applications.VGG16(include_top=False, weights='imagenet', input_tensor=Input(shape=(150,150,3)))

In [6]:
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [7]:
datagen = ImageDataGenerator(rescale=1/255.) #just normalizing data

In [8]:
generator = datagen.flow_from_directory(path + 'train/', 
                            target_size=(150,150), 
                            class_mode=None, 
                            batch_size=batch_size,
                            shuffle=False)

Found 2000 images belonging to 2 classes.


In [9]:
# bottleneck_features_train = model1.predict_generator(generator, nb_train_samples//batch_size)

In [10]:
# np.save(path + 'bottleneck_features_train.npy', bottleneck_features_train)
bottleneck_features_train = np.load(path + 'bottleneck_features_train.npy')

In [11]:
generator = datagen.flow_from_directory(path + 'validation/', 
                            target_size=(150,150), 
                            class_mode=None, 
                            batch_size=batch_size,
                            shuffle=False)

Found 800 images belonging to 2 classes.


In [12]:
# bottleneck_features_validation = model1.predict_generator(generator, nb_validation_samples//batch_size)

In [13]:
# np.save(path + 'bottleneck_features_test.npy', bottleneck_features_validation)
bottleneck_features_validation = np.load(path + 'bottleneck_features_test.npy')

Then we should train our classifier and use this bottleneck feature as its input. First we make label vectors for train and validation datasets.

In [14]:
one = np.array([0,1]*(nb_train_samples//2))
zero = np.array([1,0]*(nb_train_samples//2))
train_labels = np.concatenate([zero.reshape((nb_train_samples/2,2)), one.reshape((nb_train_samples/2,2))])

c:\users\isaac\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:3: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
one = np.array([0,1]*(nb_validation_samples//2))
zero = np.array([1,0]*(nb_validation_samples//2))
validation_labels = np.concatenate([zero.reshape(((nb_validation_samples/2),2)), one.reshape(((nb_validation_samples/2),2))])

c:\users\isaac\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:3: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  This is separate from the ipykernel package so we can avoid doing imports until


In [16]:
train_labels.shape , validation_labels.shape

((2000, 2), (800, 2))

In [17]:
bottleneck_features_train.shape , bottleneck_features_validation.shape

((2000, 4, 4, 512), (800, 4, 4, 512))

Now we build a model for our classifier: model2

In [18]:
# model2 = Sequential()
# model2.add(Flatten(input_shape=bottleneck_features_train.shape[1:]))
# model2.add(Dense(512, activation='relu'))
# model2.add(Dropout(0.5))
# model2.add(Dense(2, activation='softmax'))

In [19]:
x1 = Input(shape=bottleneck_features_train.shape[1:])
x2 = Flatten()(x1)
x3 = Dense(512, activation='relu')(x2)
x4 = Dropout(0.5)(x3)
pred = Dense(2, activation='softmax')(x4)

In [20]:
model2 = Model(inputs=x1, outputs=pred)

In [35]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 4, 4, 512)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               4194816   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 1026      
Total params: 4,195,842
Trainable params: 4,195,842
Non-trainable params: 0
_________________________________________________________________


In [21]:
model2.compile(optimizer='adam', 
               loss='categorical_crossentropy',
               metrics=['accuracy'])

In [24]:
model2.fit(bottleneck_features_train,train_labels,
           epochs = 10, 
           batch_size=batch_size,
           validation_data=(bottleneck_features_validation, validation_labels))

Train on 2000 samples, validate on 800 samples
Epoch 1/10
2000/2000 [==============================] - 2s - loss: 0.0500 - acc: 0.9835 - val_loss: 0.3717 - val_acc: 0.8888
Epoch 2/10
2000/2000 [==============================] - 2s - loss: 0.0378 - acc: 0.9855 - val_loss: 0.4224 - val_acc: 0.9050
Epoch 3/10
2000/2000 [==============================] - 2s - loss: 0.0291 - acc: 0.9910 - val_loss: 0.3952 - val_acc: 0.8850
Epoch 4/10
2000/2000 [==============================] - 2s - loss: 0.0571 - acc: 0.9765 - val_loss: 0.3994 - val_acc: 0.8925
Epoch 5/10
2000/2000 [==============================] - 2s - loss: 0.0757 - acc: 0.9705 - val_loss: 0.5113 - val_acc: 0.8862
Epoch 6/10
2000/2000 [==============================] - 2s - loss: 0.0449 - acc: 0.9805 - val_loss: 0.4273 - val_acc: 0.8938
Epoch 7/10
2000/2000 [==============================] - 2s - loss: 0.0358 - acc: 0.9865 - val_loss: 0.4202 - val_acc: 0.9012
Epoch 8/10
2000/2000 [==============================] - 2s - loss: 0.0374 - ac

In [25]:
model2.save_weights(path + 'classifier_model2.h5')

In [30]:
model = model2(model1.output)

In [31]:
tot_model = Model(inputs=model1.input, outputs=model)

In [32]:
tot_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [36]:
len(tot_model.layers)

20

In [44]:
tot_model.save_weights(path + 'tot_model_weights.h5')